<a href="https://colab.research.google.com/github/anshudaur/TensorFlowProjects/blob/master/CVDL_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#for resnet
!pip install tensorflow-probability==0.8.0rc0

     |████████████████████████████████| 2.5MB 9.3MB/s 
ERROR: tensor2tensor 1.14.1 has requirement tensorflow-probability==0.7.0, but you'll have tensorflow-probability 0.8.0rc0 which is incompatible.
ERROR: gym 0.15.4 has requirement cloudpickle~=1.2.0, but you'll have cloudpickle 1.1.1 which is incompatible.
  Found existing installation: cloudpickle 1.2.2
    Uninstalling cloudpickle-1.2.2:
      Successfully uninstalled cloudpickle-1.2.2
  Found existing installation: tensorflow-probability 0.7.0
    Uninstalling tensorflow-probability-0.7.0:
      Successfully uninstalled tensorflow-probability-0.7.0


In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
print(tf.__version__)
import pathlib
import matplotlib.pyplot as plt


TensorFlow 2.x selected.
2.0.0


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/CVDL_Project/dataset/'  #change dir to your project folder

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
data_dir=pathlib.Path(root_path+'train')
valid_dir=pathlib.Path(root_path+'valid')
test_dir=pathlib.Path(root_path+'test')
print(data_dir,',',valid_dir,',',test_dir)
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)
valid_image_count = len(list(valid_dir.glob('*/*.jpg')))
print(valid_image_count)

gdrive/My Drive/CVDL_Project/dataset/train , gdrive/My Drive/CVDL_Project/dataset/valid , gdrive/My Drive/CVDL_Project/dataset/test
4918
1638


In [4]:
import numpy as np
BATCH_SIZE = 32
IMG_HEIGHT = 224
IMG_WIDTH = 224
STEPS_PER_EPOCH = np.ceil(image_count/BATCH_SIZE)
CLASS_NAMES = np.array([item.name for item in data_dir.glob('*') if item.name != "LICENSE.txt"])
print(CLASS_NAMES)

['98' '99' '93' '97' '96' '94' '90' '95' '92' '91' '9' '89' '85' '82' '83'
 '84' '88' '86' '87' '81' '8' '80' '79' '78' '71' '72' '75' '74' '76' '73'
 '70' '77' '69' '7' '67' '68' '62' '61' '64' '63' '65' '60' '66' '6' '59'
 '58' '57' '56' '51' '49' '53' '54' '55' '52' '5' '50' '47' '48' '45' '40'
 '44' '41' '4' '43' '42' '46' '39' '38' '37' '36' '34' '31' '3' '35' '29'
 '32' '30' '33' '28' '27' '26' '25' '22' '18' '23' '21' '20' '24' '2' '19'
 '16' '17' '14' '15' '12' '11' '100' '102' '101' '1' '13' '10']


In [5]:
 image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                                                   width_shift_range=0.1,  # Horizontal shift (10% of total width)
                                                                   height_shift_range=0.1,  # Vertical shift (10% of total height)
                                                                   horizontal_flip=True,
                                                                   zoom_range=0.5,        
                                                                   rotation_range=45)
 train_data_gen = image_generator.flow_from_directory(directory=str(data_dir),
                                                      #batch_size=BATCH_SIZE,
                                                      shuffle=True,
                                                      target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                      classes = list(CLASS_NAMES),class_mode='categorical')
valid_data_gen = image_generator.flow_from_directory(directory=str(valid_dir),
                                                      #batch_size=BATCH_SIZE,
                                                      shuffle=True,
                                                      target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                      classes = list(CLASS_NAMES),class_mode='categorical')
test_data_gen = image_generator.flow_from_directory(directory=str(test_dir),
                                                      #batch_size=BATCH_SIZE,
                                                      shuffle=True,
                                                      target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                      classes = list(CLASS_NAMES),class_mode='categorical')

Found 4918 images belonging to 102 classes.
Found 1638 images belonging to 102 classes.
Found 1638 images belonging to 102 classes.


In [0]:
def show_batch(image_batch, label_batch):
  plt.figure(figsize=(10,10))
  for n in range(25):
      ax = plt.subplot(5,5,n+1)
      plt.imshow(image_batch[n])
      plt.title(CLASS_NAMES[label_batch[n]==1][0].title())
      plt.axis('off')

In [0]:
# image_batch, label_batch = next(train_data_gen)
# show_batch(image_batch, label_batch)

In [0]:
train_list_ds = tf.data.Dataset.list_files(str(data_dir/'*/*'))
test_list_ds = tf.data.Dataset.list_files(str(test_dir/'*/*'))
valid_list_ds = tf.data.Dataset.list_files(str(valid_dir/'*/*'))

print(type(train_list_ds))

<class 'tensorflow.python.data.ops.dataset_ops.DatasetV1Adapter'>


In [0]:
     
def get_label(file_path):
  # convert the path to a list of path components
  parts = tf.strings.split(file_path, os.path.sep)
  # The second to last is the class-directory
  return tf.dtypes.cast(parts[-2] == CLASS_NAMES , tf.int16)
  #one_hot_list = one_hot_encode(parts[-2], 102)
  '''if parts[-2] == CLASS_NAMES:
    label=tf.dtypes.cast(parts[-2],tf.int16)
  return tf.one_hot(label,depth=102)'''


In [0]:
def decode_img(img):
  # convert the compressed string to a 3D uint8 tensor
  img = tf.image.decode_jpeg(img, channels=3)
  # Use `convert_image_dtype` to convert to floats in the [0,1] range.
  img = tf.image.convert_image_dtype(img, tf.float32)
  # resize the image to the desired size.
  return tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT])

In [0]:
def process_path(file_path):
  label = get_label(file_path)
  print(label)
  # load the raw data from the file as a string
  img = tf.io.read_file(file_path)
  img = decode_img(img)
  return img, label

In [0]:
import os
AUTOTUNE = tf.data.experimental.AUTOTUNE
# Set `num_parallel_calls` so multiple images are loaded/processed in parallel.
train_labeled_ds = train_list_ds.map(process_path, num_parallel_calls=AUTOTUNE)
print(train_labeled_ds)
test_labeled_ds = test_list_ds.map(process_path, num_parallel_calls=AUTOTUNE)
print(test_labeled_ds)
valid_labeled_ds = valid_list_ds.map(process_path, num_parallel_calls=AUTOTUNE)
print(valid_labeled_ds)

Tensor("Cast:0", shape=(102,), dtype=int16)
<DatasetV1Adapter shapes: ((224, 224, 3), (102,)), types: (tf.float32, tf.int16)>
Tensor("Cast:0", shape=(102,), dtype=int16)
<DatasetV1Adapter shapes: ((224, 224, 3), (102,)), types: (tf.float32, tf.int16)>
Tensor("Cast:0", shape=(102,), dtype=int16)
<DatasetV1Adapter shapes: ((224, 224, 3), (102,)), types: (tf.float32, tf.int16)>


In [0]:
for image, label in train_labeled_ds.take(1):
  print("Image shape: ", image.numpy().shape)
  print("Label: ", label)

Image shape:  (224, 224, 3)
Label:  tf.Tensor(
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0], shape=(102,), dtype=int16)


In [0]:
def prepare_for_training(ds, cache=True, shuffle_buffer_size=6000):
  # This is a small dataset, only load it once, and keep it in memory.
  # use `.cache(filename)` to cache preprocessing work for datasets that don't
  # fit in memory.
  if cache:
    if isinstance(cache, str):
      ds = ds.cache(cache)
    else:
      ds = ds.cache()

  ds = ds.shuffle(buffer_size=shuffle_buffer_size)

  # Repeat forever
  ds = ds.repeat()

  ds = ds.batch(BATCH_SIZE)

  # `prefetch` lets the dataset fetch batches in the background while the model
  # is training.
  ds = ds.prefetch(buffer_size=AUTOTUNE)

  return ds

In [0]:
train_ds = prepare_for_training(train_labeled_ds)

#image_batch, label_batch = next(iter(train_ds))


In [0]:
valid_img_batch, valid_label_batch = next(iter(valid_labeled_ds))

In [0]:
show_batch(image_batch.numpy(), label_batch.numpy())

NameError: ignored

In [0]:
#for bilinearcnn
def outer_product(x):
    
    #Einstein Notation  [batch,1,1,depth] x [batch,1,1,depth] -> [batch,depth,depth]
    phi_I = tf.einsum('ijkm,ijkn->imn',x[0],x[1])

    # Reshape from [batch_size,depth,depth] to [batch_size, depth*depth]
    phi_I = tf.reshape(phi_I,[-1,x[0].shape[3]*x[1].shape[3]])
    
    # Divide by feature map size [sizexsize]
    size1 = int(x[1].shape[1])
    size2 = int(x[1].shape[2])
    phi_I = tf.divide(phi_I, size1*size2)
    
    # Take signed square root of phi_I
    y_ssqrt = tf.multiply(tf.sign(phi_I),tf.sqrt(tf.abs(phi_I)+1e-12))
    
    # Apply l2 normalization
    z_l2 = tf.nn.l2_normalize(y_ssqrt)
    return z_l2

In [0]:
from tensorflow.python.keras import backend as k
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Conv2D, BatchNormalization, Activation
from tensorflow.keras.layers import AveragePooling2D, Input, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.models import Model

from tensorflow.keras.regularizers import l2

def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr


def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder

    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)

    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x

def resnet_v2(input_shape, depth, num_classes=102):
    """ResNet Version 2 Model builder [b]

    Stacks of (1 x 1)-(3 x 3)-(1 x 1) BN-ReLU-Conv2D or also known as
    bottleneck layer
    First shortcut connection per layer is 1 x 1 Conv2D.
    Second and onwards shortcut connection is identity.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filter maps is
    doubled. Within each stage, the layers have the same number filters and the
    same filter map sizes.
    Features maps sizes:
    conv1  : 32x32,  16
    stage 0: 32x32,  64
    stage 1: 16x16, 128
    stage 2:  8x8,  256

    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)

    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 9 != 0:
        raise ValueError('depth should be 9n+2 (eg 56 or 110 in [b])')
    # Start model definition.
    num_filters_in = 16
    num_res_blocks = int((depth - 2) / 9)

    inputs = Input(shape=input_shape)
    # v2 performs Conv2D with BN-ReLU on input before splitting into 2 paths
    x = resnet_layer(inputs=inputs,
                     num_filters=num_filters_in,
                     conv_first=True)

    # Instantiate the stack of residual units
    for stage in range(3):
        for res_block in range(num_res_blocks):
            activation = 'relu'
            batch_normalization = True
            strides = 1
            if stage == 0:
                num_filters_out = num_filters_in * 4
                if res_block == 0:  # first layer and first stage
                    activation = None
                    batch_normalization = False
            else:
                num_filters_out = num_filters_in * 2
                if res_block == 0:  # first layer but not first stage
                    strides = 2    # downsample

            # bottleneck residual unit
            y = resnet_layer(inputs=x,
                             num_filters=num_filters_in,
                             kernel_size=1,
                             strides=strides,
                             activation=activation,
                             batch_normalization=batch_normalization,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_in,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_out,
                             kernel_size=1,
                             conv_first=False)
            if res_block == 0:
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters_out,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = layers.add([x, y])

        num_filters_in = num_filters_out

    # Add classifier on top.
    # v2 has BN-ReLU before Pooling
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [12]:
from tensorflow.keras import datasets, layers, models

img_shape = (224, 224, 3)


vgg_layer_list = [#layers.Conv2D(64, (3, 3), activation='relu', input_shape=(224, 224, 3)),
              #layers.Conv2D(64, (3,3), activation='relu'),
              #layers.MaxPooling2D(),
              layers.Conv2D(128, (3,3), activation='relu', input_shape=img_shape),
              layers.Conv2D(128, (3,3), activation='relu'),
              layers.Conv2D(128, (3,3), activation='relu'),
              layers.Conv2D(128, (3,3), activation='relu'),
              layers.MaxPooling2D(),
              layers.Conv2D(256, (3,3), activation='relu'),
              layers.Conv2D(256, (3,3), activation='relu'),
              layers.Conv2D(256, (3,3), activation='relu'),
              layers.Conv2D(256, (3,3), activation='relu'),
              layers.MaxPooling2D(),
              layers.Conv2D(512, (3,3), activation='relu'),
              layers.Conv2D(512, (3,3), activation='relu'),
              layers.Conv2D(512, (3,3), activation='relu'),
              layers.Conv2D(512, (3,3), activation='relu'),
              layers.MaxPooling2D(),
              layers.Conv2D(512, (3,3), activation='relu'),
              layers.Conv2D(512, (3,3), activation='relu'),
              layers.Conv2D(512, (3,3), activation='relu'),
              layers.Conv2D(512, (3,3), activation='relu')]
              #layers.MaxPooling2D(),
              #layers.Flatten(),layers.Dense(1024),layers.Dense(1024)]
              #layers.Dense(102,activation='softmax')]

#------------bilinear cnn----------------------------  
''' 
model1 = tf.keras.Sequential(vgg_layer_list)
         
d1=model1.output
d2=model1.output
print(type(d1))
print(d1.get_shape())
x = outer_product([d1,d2])
print(x.get_shape())


predictions=layers.Dense(102, activation='softmax', name='predictions')(x)

model = tf.compat.v1.keras.Model(inputs=model1.input, outputs=predictions)'''
#------------bilinear cnn---------------------------- 


#model = tf.keras.Sequential(vgg_layer_list)

#learning_rate=1e-4
#opt = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)

#--------for resnet---------------
'''model = resnet_v2(input_shape=img_shape, depth=56)
opt = tf.keras.optimizers.Adam(learning_rate=lr_schedule(0))'''
#--------for resnet---------------

'model = resnet_v2(input_shape=img_shape, depth=56)\nopt = tf.keras.optimizers.Adam(learning_rate=lr_schedule(0))'

In [0]:
def plotLosses(history):  
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()

def plotAccuracies(history):  
    plt.plot(history.history['accuracy'], label='accuracy')
    plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.ylim([0.5, 1])
    plt.legend(loc='upper right')
    plt.show()

def model2(input_shape):  
  layer_list = [layers.Conv2D(64, (3,3), activation='relu', input_shape=img_shape),
              layers.Conv2D(64, (3,3), activation='relu'),
              layers.Conv2D(64, (3,3), activation='relu'),
              layers.MaxPooling2D(),
              layers.Conv2D(128, (3,3), activation='relu'),
              layers.Conv2D(128, (3,3), activation='relu'),
              layers.Conv2D(128, (3,3), activation='relu'),
              layers.MaxPooling2D(),
              layers.Conv2D(256, (5,5), activation='relu'),
              layers.Conv2D(256, (5,5), activation='relu'),
              layers.MaxPooling2D(),
              layers.Conv2D(128, (5,5), activation='relu'),
              layers.Conv2D(128, (5,5), activation='relu'),
              layers.MaxPooling2D(),
              layers.Flatten(),layers.Dense(256,activation='softmax'),
              layers.Dropout(0.25),layers.Dense(102)]
  model = tf.keras.Sequential(layer_list)
  return model



In [0]:
import time
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.utils import multi_gpu_model
def run_model(model):
  lr_scheduler = LearningRateScheduler(lr_schedule)

  lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                                cooldown=0,
                                patience=5,
                                min_lr=0.5e-6)

  callbacks = [lr_reducer, lr_scheduler]

  start = time.time()
  parallel_model = multi_gpu_model(model, cpu_merge=False,gpus=8)
  parallel_model.compile(optimizer=Adam(learning_rate=lr_schedule(0)),
                loss='categorical_crossentropy',
                metrics=['accuracy'],callbacks=callbacks)
  parallel_model.fit_generator(train_data_gen,BATCH_SIZE=256,
                      epochs=10,steps_per_epoch = image_count//BATCH_SIZE,use_multiprocessing=True,
                      validation_data=valid_data_gen)

  plotLosses(history)
  plotAccuracies(history)
  end = time.time()
  print('Processing time:',(end - start)/60)

#model.save_weights(root_path+'resnet_cnn_model_img_generator.h5')

In [0]:
input_shape = (224, 224, 3)
#opt = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
#model = model2(input_shape,0.001)
#run_model(model)

#scores = model.evaluate_generator(test_data_gen,workers=12)
#print(scores)


In [24]:
model = resnet_v2(input_shape=input_shape, depth=56)
run_model(model)
#ResourceExhaustedError: OOM when allocating tensor with shape
#AttributeError: module 'tensorflow' has no attribute 'get_default_session'

ValueError: ignored

In [19]:

model = tf.keras.applications.densenet.DenseNet121(include_top=True, weights=None, input_tensor=None, input_shape=input_shape, pooling=None, classes=102)

run_model(model)

AttributeError: ignored